In [19]:
import numpy as np
import mediapipe as mp
import supervision as sv
from PIL import Image
from tqdm import tqdm

In [20]:
VIDEO_PATH = "example.mp4"
VIDEO_OUT = "out.mp4"
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)

In [21]:
model = mp.solutions.face_mesh.FaceMesh()
edge_annotator = sv.EdgeAnnotator(color=sv.Color.WHITE, thickness=1)

In [30]:
def process_frame(frame):
    resolution_wh = (frame.shape[1], frame.shape[0])  # Width, Height from frame
    results = model.process(frame)
    key_points = sv.KeyPoints.from_mediapipe(results, resolution_wh=resolution_wh)
    return edge_annotator.annotate(frame, key_points)

In [31]:
frame_generator = sv.get_video_frames_generator(source_path=VIDEO_PATH)

with sv.VideoSink(VIDEO_OUT, video_info) as sink:
    for frame in tqdm(frame_generator, total=video_info.total_frames):
        annotated_frame = process_frame(frame)
        sink.write_frame(annotated_frame)

100%|██████████| 510/510 [00:32<00:00, 15.86it/s]


# DataProcessor tests

In [1]:
import sys
import os

# Get the path to the src folder (assuming your notebooks and src folder are at the same level)
src_path = os.path.abspath(os.path.join('../../../', 'src'))

# Add src folder to the Python path
if src_path not in sys.path:
    sys.path.append(src_path)

# Now you can import tools
from tools.DataProcessor import DataProcessor

In [3]:
VIDEO_PATH = "example.mp4"
VIDEO_OUT = "out.mp4"

dp = DataProcessor()
dp.process_video(VIDEO_PATH)

 59%|█████▊    | 299/510 [00:22<00:15, 13.25it/s]
